## Import Libraries

In [13]:
import pandas as pd
import requests
import json
import csv
import time
import datetime
import praw


## Get list of URLs for Daily Discussion Threads

In [14]:
# Adapted from https://medium.com/@RareLoot/using-pushshifts-api-to-extract-reddit-submissions-fb517b286563

def getPushshiftData(query, after, before, sub):
    url = 'https://api.pushshift.io/reddit/search/submission/?title='+str(query)+'&size=1000&after='+str(after)+'&before='+str(before)+'&subreddit='+str(sub)
    r = requests.get(url)
    data = json.loads(r.text)
    return data['data']

def collectSubData(subm):
    subData = [subm['title'], 
               subm['url'], 
               subm['created_utc'], 
               datetime.datetime.fromtimestamp(subm['created_utc']).date()]
    subStats.append(subData)


In [15]:
# Subreddit to query
sub='wallstreetbets'
query = "Daily Discussion Thread -"
# Dates
# after = "1561939200"  # July 1st, 2019
# before = "1593561600" # July 1st, 2020
after = "1530403200" # July 1st, 2018
before = "1561939200" # July 1st, 2019
subStats = []


In [16]:
data = getPushshiftData(query, after, before, sub)
while len(data) > 0:
    for submission in data:
        collectSubData(submission)
    after = data[-1]['created_utc']
    data = getPushshiftData(query, after, before, sub)
    

In [17]:
data = {}
posts = []
urls = []
dates = []
dates_convert = []
for i in subStats:
    if i[3] in dates_convert:
        # Occasionally there are posts about the discussion thread after it's posted
        # and its better to ignore those posts
        pass
    else:
        posts.append(i[0])
        urls.append(i[1])
        dates.append(i[2])
        dates_convert.append(i[3])
data['posts'] = posts
data['urls'] = urls
data['date'] = dates
data['date_convert'] = dates_convert
list_posts = pd.DataFrame(data)


In [18]:
list_posts


,posts,urls,date,date_convert
0,"Daily Discussion Thread - July 02, 2018",https://www.reddit.com/r/wallstreetbets/commen...,1530533389,2018-07-02
1,"Daily Discussion Thread - July 03, 2018",https://www.reddit.com/r/wallstreetbets/commen...,1530619650,2018-07-03
2,"Daily Discussion Thread - July 04, 2018",https://www.reddit.com/r/wallstreetbets/commen...,1530706128,2018-07-04
3,"Daily Discussion Thread - July 05, 2018",https://www.reddit.com/r/wallstreetbets/commen...,1530792461,2018-07-05
4,"Daily Discussion Thread - July 06, 2018",https://www.reddit.com/r/wallstreetbets/commen...,1530878916,2018-07-06
...,...,...,...,...
254,"Daily Discussion Thread - June 24, 2019",https://www.reddit.com/r/wallstreetbets/commen...,1561374410,2019-06-24
255,"Daily Discussion Thread - June 25, 2019",https://www.reddit.com/r/wallstreetbets/commen...,1561460841,2019-06-25
256,"Daily Discussion Thread - June 26, 2019",https://www.reddit.com/r/wallstreetbets/commen...,1561547195,2019-06-26
257,"Daily Discussion Thread - June 27, 2019",https://www.reddit.com/r/wallstreetbets/commen...,1561633557,2019-06-27


In [19]:
list_urls = list(list_posts['urls'])
list_urls


['https://www.reddit.com/r/wallstreetbets/comments/8vi4ew/daily_discussion_thread_july_02_2018/',
 'https://www.reddit.com/r/wallstreetbets/comments/8vrnx1/daily_discussion_thread_july_03_2018/',
 'https://www.reddit.com/r/wallstreetbets/comments/8w15qq/daily_discussion_thread_july_04_2018/',
 'https://www.reddit.com/r/wallstreetbets/comments/8w9x77/daily_discussion_thread_july_05_2018/',
 'https://www.reddit.com/r/wallstreetbets/comments/8wjjlf/daily_discussion_thread_july_06_2018/',
 'https://www.reddit.com/r/wallstreetbets/comments/8xaznc/daily_discussion_thread_july_09_2018/',
 'https://www.reddit.com/r/wallstreetbets/comments/8xofw3/daily_discussion_thread_july_10_2018/',
 'https://www.reddit.com/r/wallstreetbets/comments/8xynz4/daily_discussion_thread_july_11_2018/',
 'https://www.reddit.com/r/wallstreetbets/comments/8y8tlt/daily_discussion_thread_july_12_2018/',
 'https://www.reddit.com/r/wallstreetbets/comments/8yj2pc/daily_discussion_thread_july_13_2018/',
 'https://www.reddit

In [8]:
# Remove unrelated link that isn't a discussion thread
list_urls.remove('https://i.redd.it/ezmiqy4icl551.jpg')
list_urls


['https://www.reddit.com/r/wallstreetbets/comments/c7r8d1/daily_discussion_thread_july_01_2019/',
 'https://www.reddit.com/r/wallstreetbets/comments/c87x3r/daily_discussion_thread_july_02_2019/',
 'https://www.reddit.com/r/wallstreetbets/comments/c8mrrx/daily_discussion_thread_july_03_2019/',
 'https://www.reddit.com/r/wallstreetbets/comments/c91b7h/daily_discussion_thread_july_04_2019/',
 'https://www.reddit.com/r/wallstreetbets/comments/c9epea/daily_discussion_thread_july_05_2019/',
 'https://www.reddit.com/r/wallstreetbets/comments/cajnty/daily_discussion_thread_july_08_2019/',
 'https://www.reddit.com/r/wallstreetbets/comments/cazfen/daily_discussion_thread_july_09_2019/',
 'https://www.reddit.com/r/wallstreetbets/comments/cbfc3z/daily_discussion_thread_july_10_2019/',
 'https://www.reddit.com/r/wallstreetbets/comments/cbuf6d/daily_discussion_thread_july_11_2019/',
 'https://www.reddit.com/r/wallstreetbets/comments/cc9g83/daily_discussion_thread_july_12_2019/',
 'https://www.reddit

## Get Comments from each Daily Discussion Thread

In [20]:
# Adapted from https://towardsdatascience.com/scraping-reddit-data-1c0af3040768

# Created throwaway Reddit account
reddit = praw.Reddit(client_id='wThKpRFjtgwSug', client_secret='MspknE5sxHRJHX4IHazZeJmkrio', user_agent='nlp_wallstreetbets')

data = {}
comments = []
scores = []
dates = []
url_links = []
url_dates = []
for url in list_urls:
    while True:
        try:
            url_date = int(list_posts.loc[list_posts['urls'] == url]['date'])
            submission = reddit.submission(url=url)
            submission.comments.replace_more(limit=0)

            comments.extend(list([(comment.body) for comment in submission.comments])) # comment text
            scores.extend(list([(comment.score) for comment in submission.comments])) # comment score
            dates.extend(list([(comment.created_utc) for comment in submission.comments])) # comment creation time
            url_links.extend([url] * len(list([(comment.body) for comment in submission.comments]))) # post link
            url_dates.extend([url_date] * len(list([(comment.body) for comment in submission.comments]))) # post creation

            print(url)
            time.sleep(30)
        except:
            # Occasionally the Reddit API times-out
            # so this exception waits and then repeats the request that timed-out
            print('***** TIMEOUT ERROR *****')
            time.sleep(60)
            continue
        break

data['comment'] = comments
data['score'] = scores
data['date'] = dates
data['url_link'] = url_links
data['url_date'] = url_dates
list_comments = pd.DataFrame(data)


https://www.reddit.com/r/wallstreetbets/comments/8vi4ew/daily_discussion_thread_july_02_2018/
https://www.reddit.com/r/wallstreetbets/comments/8vrnx1/daily_discussion_thread_july_03_2018/
https://www.reddit.com/r/wallstreetbets/comments/8w15qq/daily_discussion_thread_july_04_2018/
https://www.reddit.com/r/wallstreetbets/comments/8w9x77/daily_discussion_thread_july_05_2018/
https://www.reddit.com/r/wallstreetbets/comments/8wjjlf/daily_discussion_thread_july_06_2018/
https://www.reddit.com/r/wallstreetbets/comments/8xaznc/daily_discussion_thread_july_09_2018/
https://www.reddit.com/r/wallstreetbets/comments/8xofw3/daily_discussion_thread_july_10_2018/
https://www.reddit.com/r/wallstreetbets/comments/8xynz4/daily_discussion_thread_july_11_2018/
https://www.reddit.com/r/wallstreetbets/comments/8y8tlt/daily_discussion_thread_july_12_2018/
https://www.reddit.com/r/wallstreetbets/comments/8yj2pc/daily_discussion_thread_july_13_2018/
https://www.reddit.com/r/wallstreetbets/comments/8za1fy/dail

https://www.reddit.com/r/wallstreetbets/comments/9sc3oa/daily_discussion_thread_october_29_2018/
https://www.reddit.com/r/wallstreetbets/comments/9smtdh/daily_discussion_thread_october_30_2018/
https://www.reddit.com/r/wallstreetbets/comments/9sxovr/daily_discussion_thread_october_31_2018/
https://www.reddit.com/r/wallstreetbets/comments/9t8fww/daily_discussion_thread_november_01_2018/
https://www.reddit.com/r/wallstreetbets/comments/9tj1rn/daily_discussion_thread_november_02_2018/
https://www.reddit.com/r/wallstreetbets/comments/9ucmkq/daily_discussion_thread_november_05_2018/
https://www.reddit.com/r/wallstreetbets/comments/9unf92/daily_discussion_thread_november_06_2018/
https://www.reddit.com/r/wallstreetbets/comments/9uyidt/daily_discussion_thread_november_07_2018/
https://www.reddit.com/r/wallstreetbets/comments/9v936h/daily_discussion_thread_november_08_2018/
https://www.reddit.com/r/wallstreetbets/comments/9vjjyh/daily_discussion_thread_november_09_2018/
https://www.reddit.com/

https://www.reddit.com/r/wallstreetbets/comments/avcaso/daily_discussion_thread_february_27_2019/
https://www.reddit.com/r/wallstreetbets/comments/avr4tf/daily_discussion_thread_february_28_2019/
https://www.reddit.com/r/wallstreetbets/comments/aw3n6d/daily_discussion_thread_march_01_2019/
https://www.reddit.com/r/wallstreetbets/comments/ax5xn0/daily_discussion_thread_march_04_2019/
https://www.reddit.com/r/wallstreetbets/comments/axjyk3/daily_discussion_thread_march_05_2019/
https://www.reddit.com/r/wallstreetbets/comments/axxnbq/daily_discussion_thread_march_06_2019/
https://www.reddit.com/r/wallstreetbets/comments/aybghn/daily_discussion_thread_march_07_2019/
https://www.reddit.com/r/wallstreetbets/comments/ayp28a/daily_discussion_thread_march_08_2019/
https://www.reddit.com/r/wallstreetbets/comments/azs500/daily_discussion_thread_march_11_2019/
https://www.reddit.com/r/wallstreetbets/comments/b06hok/daily_discussion_thread_march_12_2019/
https://www.reddit.com/r/wallstreetbets/comm

https://www.reddit.com/r/wallstreetbets/comments/c6iv89/daily_discussion_thread_june_28_2019/


In [21]:
list_comments


,comment,score,date,url_link,url_date
0,[deleted],48,1.530534e+09,https://www.reddit.com/r/wallstreetbets/commen...,1530533389
1,It's only a loss if you sell.\n\nIt's only a l...,36,1.530540e+09,https://www.reddit.com/r/wallstreetbets/commen...,1530533389
2,Down 11% pretty much immediately.\n\nCalls are...,21,1.530540e+09,https://www.reddit.com/r/wallstreetbets/commen...,1530533389
3,I've lost too much money buying the last dips ...,35,1.530537e+09,https://www.reddit.com/r/wallstreetbets/commen...,1530533389
4,Is this it? Is this the end? I never told that...,57,1.530536e+09,https://www.reddit.com/r/wallstreetbets/commen...,1530533389
...,...,...,...,...,...
82016,$KAR down -61% oooooffffff,3,1.561734e+09,https://www.reddit.com/r/wallstreetbets/commen...,1561719981
82017,When SPY 300?,3,1.561734e+09,https://www.reddit.com/r/wallstreetbets/commen...,1561719981
82018,https://i.imgur.com/PbGHW3N.jpg \n\nPool and B...,3,1.561734e+09,https://www.reddit.com/r/wallstreetbets/commen...,1561719981
82019,"Guys $KAR Is going to moon, down 61% on like 2...",3,1.561734e+09,https://www.reddit.com/r/wallstreetbets/commen...,1561719981


In [22]:
list_comments.to_pickle('./list_comments.pkl')
#list_comments = pd.read_pickle('./list_comments.pkl')
